In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [9]:
data=pd.read_csv("Churn_Modelling.csv")

In [10]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
##preprocess the data drop irrelevant features



In [15]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [16]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
gender_encoder=LabelEncoder()
data['Gender']=gender_encoder.fit_transform(data['Gender'])

In [21]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [24]:
ONE_HOT=OneHotEncoder()
encoder_geo=ONE_HOT.fit_transform(data[['Geography']])
encoder_geo

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [29]:
encoder_geo.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [31]:
column=ONE_HOT.get_feature_names_out(['Geography'])

In [34]:
geo_df=pd.DataFrame(encoder_geo.toarray(),columns=column)

In [43]:
data=pd.concat([data.drop(['Geography'],axis=1),geo_df],axis=1)

In [44]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [45]:
## save the encoder and scaler
with open('gender_encoder','wb') as file:
    pickle.dump(gender_encoder,file)

with open('ONE_HOT','wb') as file:
    pickle.dump(ONE_HOT,file)

In [50]:
## divide the dataset into independent and dependent features

X=data.drop(['Exited'],axis=1)
y=data['Exited']

In [49]:
## divide the data into training and testing dataset

In [81]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [82]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9069,619,0,32,4,175406.13,2,1,1,172792.43,0.0,0.0,1.0
2603,643,0,34,7,160426.07,1,0,1,188533.11,0.0,1.0,0.0
7738,561,1,33,6,0.00,2,0,0,173680.39,1.0,0.0,0.0
1579,618,1,41,8,37702.79,1,1,1,195775.48,0.0,1.0,0.0
5058,714,1,37,9,148466.93,2,0,1,151280.96,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [83]:
## scale these featues
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)





In [58]:
with open('scaler','wb') as file:
    pickle.dump(scaler,file)

In [59]:
# ANN IMPLEMENTATION

In [60]:
import tensorflow as tf

In [61]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [64]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [63]:
##build the ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ##hidden layer 1
    Dense(32,activation='relu'), ## hidden layer 2
    Dense(1,activation='sigmoid')  ## output
    
])

In [85]:
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [66]:
## set up the tensorBoard

log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [67]:
## set up the early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [86]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 23s 96ms/step - accuracy: 0.8062 - loss: 0.4488 - val_accuracy: 0.8597 - val_loss: 0.3495
Epoch 2/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8443 - loss: 0.3759 - val_accuracy: 0.8633 - val_loss: 0.3480
Epoch 3/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8566 - loss: 0.3479 - val_accuracy: 0.8653 - val_loss: 0.3438
Epoch 4/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8563 - loss: 0.3428 - val_accuracy: 0.8647 - val_loss: 0.3368
Epoch 5/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8686 - loss: 0.3327 - val_accuracy: 0.8677 - val_loss: 0.3373
Epoch 6/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8645 - loss: 0.3383 - val_accuracy: 0.8643 - val_loss: 0.3409
Epoch 7/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8604 - loss: 0.3326 - val_accuracy: 0.8623 - val_loss: 0.3402
Epoch 8/100
219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8622 - loss: 0.3359 - val_ac

In [76]:
print(type(X_train), X_train.shape)
print(type(y_train), y_train.shape)


<class 'numpy.ndarray'> (7000, 12)
<class 'pandas.core.series.Series'> (7000,)


In [87]:
model.save('model.h5')

In [90]:
# import tensorboard

%load_ext tensorboard


In [93]:
%tensorboard --logdir logs/fit20250130-205601

Reusing TensorBoard on port 6007 (pid 4836), started 0:02:27 ago. (Use '!kill 4836' to kill it.)